In [1]:
# gets all this setup
import time

start_time = time.time()

from transformers import BertTokenizer
from pathlib import Path
import torch

from box import Box
import pandas as pd
import collections
import os
from tqdm import tqdm, trange
import sys
import random
import numpy as np
# import apex
from sklearn.model_selection import train_test_split

import datetime

import sys
sys.path.append('../')

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, \
    convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import *

for column in ["is_unemployed", "lost_job_1mo", "job_search", "is_hired_1mo", "job_offer"]:

    print(column, 'creating model and loading..')

    torch.cuda.empty_cache()

    pd.set_option('display.max_colwidth', -1)
    run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

    if not os.path.exists('/scratch/da2734/twitter/mturk_mar6/log_{}/'.format(column)):
        os.makedirs('/scratch/da2734/twitter/mturk_mar6/log_{}/'.format(column))

    if not os.path.exists('/scratch/da2734/twitter/mturk_mar6/output_{}'.format(column)):
        os.makedirs('/scratch/da2734/twitter/mturk_mar6/output_{}'.format(column))

    LOG_PATH = Path('/scratch/da2734/twitter/mturk_mar6/log_{}/'.format(column))
    DATA_PATH = Path('/scratch/da2734/twitter/mturk_mar6/data_binary/')
    LABEL_PATH = Path('/scratch/da2734/twitter/mturk_mar6/data_binary/')
    OUTPUT_PATH = Path('/scratch/da2734/twitter/mturk_mar6/output_{}'.format(column))
    FINETUNED_PATH = None

    args = Box({
        "run_text": "labor mturk ar 6 binary",
        "train_size": -1,
        "val_size": -1,
        "log_path": LOG_PATH,
        "full_data_dir": DATA_PATH,
        "data_dir": DATA_PATH,
        "task_name": "labor_market_classification",
        "no_cuda": False,
        #     "bert_model": BERT_PRETRAINED_PATH,
        "output_dir": OUTPUT_PATH,
        "max_seq_length": 512,
        "do_train": True,
        "do_eval": True,
        "do_lower_case": True,
        "train_batch_size": 8,
        "eval_batch_size": 16,
        "learning_rate": 5e-5,
        "num_train_epochs": 10,
        "warmup_proportion": 0.0,
        "no_cuda": False,
        "local_rank": -1,
        "seed": 42,
        "gradient_accumulation_steps": 1,
        "optimize_on_cpu": False,
        "fp16": False,
        "fp16_opt_level": "O1",
        "weight_decay": 0.0,
        "adam_epsilon": 1e-8,
        "max_grad_norm": 1.0,
        "max_steps": -1,
        "warmup_steps": 500,
        "logging_steps": 50,
        "eval_all_checkpoints": True,
        "overwrite_output_dir": True,
        "overwrite_cache": False,
        "seed": 42,
        "loss_scale": 128,
        "task_name": 'intent',
        "model_name": 'bert-base-uncased',
        "model_type": 'bert'
    })

    import logging

    logfile = str(LOG_PATH / 'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
        datefmt='%m/%d/%Y %H:%M:%S',
        handlers=[
            logging.FileHandler(logfile),
            logging.StreamHandler(sys.stdout)
        ])

    logger = logging.getLogger()

    logger.info(args)

    device = torch.device('cuda')
    if torch.cuda.device_count() > 1:
        args.multi_gpu = True
    else:
        args.multi_gpu = False

    # label_cols = ["job_loss","is_unemployed","job_search","is_hired","job_offer"]
    label_cols = [column]

    databunch = BertDataBunch(
        args['data_dir'],
        LABEL_PATH,
        args.model_name,
        train_file='train_{}.csv'.format(column),
        val_file='val_{}.csv'.format(column),
        label_file='label_{}.csv'.format(column),
        # test_data='test.csv',
        text_col="text",  # this is the name of the column in the train file that containts the tweet text
        label_col=label_cols,
        batch_size_per_gpu=args['train_batch_size'],
        max_seq_length=args['max_seq_length'],
        multi_gpu=args.multi_gpu,
        multi_label=True,
        model_type=args.model_type)

    num_labels = len(databunch.labels)
    print('num_labels', num_labels)

    print('time taken to load all this stuff:', str(time.time() - start_time), 'seconds')
    
    # metrics defined: https://github.com/kaushaltrivedi/fast-bert/blob/d89e2aa01d948d6d3cdea7ad106bf5792fea7dfa/fast_bert/metrics.py
    metrics = []
    metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
    metrics.append({'name': 'roc_auc', 'function': roc_auc})
    metrics.append({'name': 'fbeta', 'function': fbeta})
    metrics.append({'name': 'accuracy', 'function': accuracy})
    metrics.append({'name': 'accuracy_multilabel', 'function': accuracy_multilabel})

    learner = BertLearner.from_pretrained_model(
                                            databunch, 
                                            pretrained_path=args.model_name, 
                                            metrics=metrics, 
                                            device=device, 
                                            logger=logger, 
                                            output_dir=args.output_dir, 
                                            finetuned_wgts_path=FINETUNED_PATH, 
                                            warmup_steps=args.warmup_steps,
                                            multi_gpu=args.multi_gpu, 
                                            is_fp16=args.fp16, 
                                            multi_label=True, 
                                            logging_steps=0)

    break

is_unemployed creating model and loading..
03/29/2020 16:19:08 - INFO - root -   {'run_text': 'labor mturk ar 6 binary', 'train_size': -1, 'val_size': -1, 'log_path': PosixPath('/scratch/da2734/twitter/mturk_mar6/log_is_unemployed'), 'full_data_dir': PosixPath('/scratch/da2734/twitter/mturk_mar6/data_binary'), 'data_dir': PosixPath('/scratch/da2734/twitter/mturk_mar6/data_binary'), 'task_name': 'intent', 'no_cuda': False, 'output_dir': PosixPath('/scratch/da2734/twitter/mturk_mar6/output_is_unemployed'), 'max_seq_length': 512, 'do_train': True, 'do_eval': True, 'do_lower_case': True, 'train_batch_size': 8, 'eval_batch_size': 16, 'learning_rate': 5e-05, 'num_train_epochs': 10, 'warmup_proportion': 0.0, 'local_rank': -1, 'seed': 42, 'gradient_accumulation_steps': 1, 'optimize_on_cpu': False, 'fp16': False, 'fp16_opt_level': 'O1', 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'max_steps': -1, 'warmup_steps': 500, 'logging_steps': 50, 'eval_all_checkpoints': True, 'over

/home/da2734/miniconda3/envs/worldbank/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


03/29/2020 16:19:10 - INFO - root -   Writing example 0 of 4407
03/29/2020 16:19:13 - INFO - root -   Saving features into cached file /scratch/da2734/twitter/mturk_mar6/data_binary/cache/cached_bert_train_multi_label_512_train_is_unemployed.csv
03/29/2020 16:19:16 - INFO - root -   Writing example 0 of 1101
03/29/2020 16:19:17 - INFO - root -   Saving features into cached file /scratch/da2734/twitter/mturk_mar6/data_binary/cache/cached_bert_dev_multi_label_512_val_is_unemployed.csv
num_labels 2
time taken to load all this stuff: 11.903232336044312 seconds


In [2]:
databunch.train_dl.dataset[2][3] # this train_dlgives us the training dataset for example 2's labels

tensor([1.])

In [3]:
# metrics defined: https://github.com/kaushaltrivedi/fast-bert/blob/d89e2aa01d948d6d3cdea7ad106bf5792fea7dfa/fast_bert/metrics.py
metrics = []
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})
metrics.append({'name': 'accuracy', 'function': accuracy})
metrics.append({'name': 'accuracy_multilabel', 'function': accuracy_multilabel})


In [5]:
learner = BertLearner.from_pretrained_model(
                                            databunch, 
                                            pretrained_path=args.model_name, 
                                            metrics=metrics, 
                                            device=device, 
                                            logger=logger, 
                                            output_dir=args.output_dir, 
                                            finetuned_wgts_path=FINETUNED_PATH, 
                                            warmup_steps=args.warmup_steps,
                                            multi_gpu=args.multi_gpu, 
                                            is_fp16=args.fp16, 
                                            multi_label=True, 
                                            logging_steps=0)

03/29/2020 16:21:16 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/da2734/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
03/29/2020 16:21:16 - INFO - transformers.configuration_utils -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "outpu

In [6]:
learner.fit(args.num_train_epochs, args.learning_rate, validate=True) #this trains the model

03/29/2020 16:21:26 - INFO - root -   ***** Running training *****
03/29/2020 16:21:26 - INFO - root -     Num examples = 4407
03/29/2020 16:21:26 - INFO - root -     Num Epochs = 10
03/29/2020 16:21:26 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
03/29/2020 16:21:26 - INFO - root -     Gradient Accumulation steps = 1
03/29/2020 16:21:26 - INFO - root -     Total optimization steps = 5510


saving to file
/scratch/da2734/twitter/mturk_mar6/output_is_unemployed/model_out_0
03/29/2020 16:21:26 - INFO - transformers.configuration_utils -   Configuration saved in /scratch/da2734/twitter/mturk_mar6/output_is_unemployed/model_out_0/config.json
03/29/2020 16:21:27 - INFO - transformers.modeling_utils -   Model weights saved in /scratch/da2734/twitter/mturk_mar6/output_is_unemployed/model_out_0/pytorch_model.bin


ValueError: Target size (torch.Size([4, 2])) must be the same as input size (torch.Size([8, 2]))

In [5]:
learner.validate()

03/06/2020 23:41:11 - INFO - root -   Running evaluation
03/06/2020 23:41:11 - INFO - root -     Num examples = 737
03/06/2020 23:41:11 - INFO - root -     Batch size = 16


{'loss': 0.23552671804073008,
 'accuracy_thresh': 0.9175034165382385,
 'roc_auc': 0.8943037311014089,
 'fbeta': 0.1971413791179657,
 'accuracy': 0.0,
 'accuracy_multilabel': 0.6390773405698779}

In [6]:
learner.save_model()

03/06/2020 23:41:38 - INFO - transformers.configuration_utils -   Configuration saved in /scratch/da2734/twitter/mturk_mar6/output_100/model_out/config.json
03/06/2020 23:41:41 - INFO - transformers.modeling_utils -   Model weights saved in /scratch/da2734/twitter/mturk_mar6/output_100/model_out/pytorch_model.bin


In [7]:
texts = ['I just received a job offer']
predictions = learner.predict_batch(texts)
print(predictions[0])

03/06/2020 23:41:52 - INFO - root -   Writing example 0 of 1
[('is_hired_1mo', 0.35232582688331604), ('lost_job_1mo', 0.31937041878700256), ('is_unemployed', 0.31286755204200745), ('job_search', 0.23360265791416168), ('job_offer"', 0.056336939334869385)]
